### Import Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import numpy as np
import random

### Define Functions

In [2]:
def scrape_manga(res, mangas):
    soup = BeautifulSoup(res.text, 'lxml')
    table = soup.find('table', {'class': 'top-ranking-table'})
    titles = table.find_all('a', {'class': 'hoverinfo_trigger fs14 fw-b'})

    for title in titles:
        mangas['manga_title'].append(title.text)
        mangas['manga_link'].append(title.attrs['href'])
    
    return mangas



def scrape_user_stats_from_one_manga(res, reviews, title):
    soup = BeautifulSoup(res.text, 'lxml')
    table = soup.find('table', {'class': 'table-recently-updated'})
    table_rows = table.find_all('tr')

    for row in table_rows[1:]:
        reviews['title'].append(title)
        reviews['member'].append(row.find_all('td')[0].find_all('a')[1].text)
        reviews['member_link'].append(row.find_all('td')[0].find_all('a')[1].attrs['href'])
        reviews['score'].append(row.find_all('td')[1].text)
        reviews['status'].append(row.find_all('td')[2].text)
        reviews['vol_read'].append(row.find_all('td')[3].text.replace('\n','').strip())
        reviews['chpt_read'].append(row.find_all('td')[4].text.replace('\n','').strip())
    
    return reviews


def scrape_one_manga_details(res):
    manga_details = {}
    soup = BeautifulSoup(res.text, 'lxml')
    table = soup.find_all('div', {'class': 'spaceit_pad'})
    for row in table:
        spans = row.find_all('span')
        if len(spans) == 1:
            result = re.search(':', row.text)
            if result:
                manga_details[row.text[0: result.span()[0]].strip()] = row.text[result.span()[1]+1:].strip()
        else:
            group = ""
            for item in spans[1:]:
                group = group + item.text + ", "
            group = group[:-2]
            manga_details[spans[0].text[:-1].strip()] = group
            
    return manga_details


### Webscrap Titles From Top Manga

In [82]:
# Initialize list
mangas = {'manga_title':list(), 'manga_link':list()}

# No of pages to comb through
no_of_pages = 250
no_of_titles_per_page = 50

# Define base url to search for the manga
base_url = "https://myanimelist.net/topmanga.php"

# Loop through the top number of pages to get the list of manga title and link
for i in range(no_of_pages):
    # Set origin url
    url = base_url + '?type=manga&limit=' + str(i*no_of_titles_per_page)
    
    # Set initial status
    status = 0
    
    while status != 200:
        res = requests.get(url)
        status = res.status_code
        if status == 200:
            scrape_manga(res, mangas)
            print(f'Scraped Page {i}')
        elif status == 404:
            print(f"{status} occurred at index {i}")
            break
        else:
            print(str(i) + "_" + 'others')
        
        time.sleep(5)
        
# Transform dictionary to DataFrame
mangas_pd = pd.DataFrame([mangas['manga_title'],
                           mangas['manga_link'],]).T
mangas_pd.columns = ['manga_title', 'manga_link']

Scraped Page 0
Scraped Page 1
Scraped Page 2
Scraped Page 3
Scraped Page 4
Scraped Page 5
Scraped Page 6
Scraped Page 7
Scraped Page 8
Scraped Page 9
Scraped Page 10
Scraped Page 11
Scraped Page 12
Scraped Page 13
Scraped Page 14
Scraped Page 15
Scraped Page 16
Scraped Page 17
Scraped Page 18
Scraped Page 19
Scraped Page 20
Scraped Page 21
Scraped Page 22
Scraped Page 23
Scraped Page 24
Scraped Page 25
Scraped Page 26
Scraped Page 27
Scraped Page 28
Scraped Page 29
Scraped Page 30
Scraped Page 31
Scraped Page 32
Scraped Page 33
Scraped Page 34
Scraped Page 35
Scraped Page 36
Scraped Page 37
Scraped Page 38
Scraped Page 39
Scraped Page 40
Scraped Page 41
Scraped Page 42
Scraped Page 43
Scraped Page 44
Scraped Page 45
Scraped Page 46
Scraped Page 47
Scraped Page 48
Scraped Page 49
Scraped Page 50
Scraped Page 51
Scraped Page 52
Scraped Page 53
Scraped Page 54
Scraped Page 55
Scraped Page 56
Scraped Page 57
Scraped Page 58
Scraped Page 59
Scraped Page 60
Scraped Page 61
Scraped Page 62
Sc

In [83]:
mangas_pd.to_csv('../data/manga_titles.csv')

### Webscrape Title Details For Each Series

In [19]:
# Read manga list from file
manga_list = pd.read_csv("../data/manga_titles.csv")
manga_list.drop(columns=['Unnamed: 0'], inplace=True)
manga_list

,manga_title,manga_link
0,Berserk,https://myanimelist.net/manga/2/Berserk
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,https://myanimelist.net/manga/1706/JoJo_no_Kim...
2,One Piece,https://myanimelist.net/manga/13/One_Piece
3,Vagabond,https://myanimelist.net/manga/656/Vagabond
4,Monster,https://myanimelist.net/manga/1/Monster
...,...,...
12495,Shiniden,https://myanimelist.net/manga/2544/Shiniden
12496,Tenshi ni Naru mon!,https://myanimelist.net/manga/2569/Tenshi_ni_N...
12497,Zenmaijikake no Tina,https://myanimelist.net/manga/2582/Zenmaijikak...
12498,Niji no Densetsu,https://myanimelist.net/manga/2586/Niji_no_Den...


In [40]:
for loop in range(97, int(manga_list.shape[0]/100), 1):
    print(f"Loop {loop}")
    mangas_details = {}
    start = 0 + (loop*100)
    end = 100 + (loop*100)
    #end = manga_list.shape[0]
    for row in range(start, end, 1):
        # Set base_url to manga page
        url = manga_list.loc[row, 'manga_link'] + "/stats?show=0"

        # Set initial status
        status = 0

        # While loop until status is 200 or until 404 occurs
        while status != 200:
            res = requests.get(url, headers = {"HTTP_HOST": "MyVeryOwnHost"})
            status = res.status_code
            if status == 200:
                mangas_details[manga_list.index[row]] = scrape_one_manga_details(res)
                mangas_details[manga_list.index[row]]['manga_title'] = manga_list.loc[row, 'manga_title']
                print(f"Scraped Title {row}/{end}: {manga_list.loc[row, 'manga_title']}")
            elif status == 404:
                print(f"Code {status} occurred at index {row}")
                break
            else:
                print(str(row) + "_" + 'others')
            time.sleep(random.random() * 5)

    # Initialize mangas_info
    mangas_info = {'index': list(),
                   'title': list(),
                   'volumes': list(),
                   'chapters': list(),
                   'status': list(),
                   'published': list(),
                   'genres': list(),
                   'themes': list(),
                   'demographic': list(),
                   'serialization': list(),
                   'authors': list()}

    # Define the dest and source pairing
    pairs = [('title', 'manga_title'),
             ('volumes', 'Volume'),
             ('chapters', 'Chapter'),
             ('status', 'Status'),
             ('published', 'Published'),
             ('genres', 'Genre'),
             ('themes', 'Theme'),
             ('demographic', 'Demographic'),
             ('serialization', 'Serialization'),
             ('authors', 'Authors')]

    # Extract features of interest into respective dictionary: list
    for index, manga_detail in mangas_details.items():
        # Get title and append to list under mangas_info['title']
        mangas_info['index'].append(index)
        for dest, source in pairs:
            try:
                mangas_info[dest].append(manga_detail[source])
            except:
                try:
                    mangas_info[dest].append(manga_detail[source+"s"])
                except:
                    mangas_info[dest].append(np.nan)

    # Convert dict to dataframe
    mangas_info_df = pd.DataFrame.from_dict(mangas_info)

    # Write to csv
    mangas_info_df.to_csv(f'../data/manga_details/manga_details_' + str(loop) + '.csv')

Loop 97
Scraped Title 9700/9800: Heroine Voice
Scraped Title 9701/9800: Versailles of the Dead
Scraped Title 9702/9800: Yu☆Gi☆Oh!: Transcend Game
Scraped Title 9703/9800: Buster Dress
Scraped Title 9704/9800: Kaijuu no Buki Shokunin
Scraped Title 9705/9800: Mainichi Kiss shite Ii desu ka?
Scraped Title 9706/9800: Hirune Hime: Shiranai Watashi no Monogatari
Scraped Title 9707/9800: Eguchi-kun wa Minogasanai
Scraped Title 9708/9800: Nariyuki Makase no Tengu Michi
Scraped Title 9709/9800: Koisuru Herb Shohousen
Scraped Title 9710/9800: Oto☆Koi
Scraped Title 9711/9800: Chikai no Kiss wa, 16-sai
Scraped Title 9712/9800: Kono Sekai ga Game da to Ore dake ga Shitteiru
Scraped Title 9713/9800: Kagamigami
Scraped Title 9714/9800: Kimi to Kyun Koi, Shiyou.
Scraped Title 9715/9800: Oooku no Sakura: Gendai Oooku Jogakuin
Scraped Title 9716/9800: Dorothy wa Gokigennaname?
Scraped Title 9717/9800: Shiroi Majo
Scraped Title 9718/9800: Hajimari wa Suna no Ichiya
Scraped Title 9719/9800: Higan no Ishi


Scraped Title 9868/9900: Erotic Horror
Scraped Title 9869/9900: Pokémon Get da ze!
Scraped Title 9870/9900: Houjin Exaxxion
Scraped Title 9871/9900: Medarot
Scraped Title 9872/9900: .hack//Alcor: Hagun no Jokyoku
Scraped Title 9873/9900: Maya no Souretsu
Scraped Title 9874/9900: Dragonaut: The Resonance
Scraped Title 9875/9900: BX
Scraped Title 9876/9900: Extra Heavy Syrup
Scraped Title 9877/9900: Ramen Ikaga!?
Scraped Title 9878/9900: Masuraou
Scraped Title 9879/9900: Mach GoGoGo
Scraped Title 9880/9900: Toki☆Meka!
Scraped Title 9881/9900: Koko ni Iru Suiren
Scraped Title 9882/9900: Oniichan no Koto nanka Zenzen Suki ja Nai n da kara ne!!
Scraped Title 9883/9900: Canaria Child
Scraped Title 9884/9900: Aishi Aisare.
Scraped Title 9885/9900: Hokuto no Ken - Juuza Gaiden
Scraped Title 9886/9900: Goodbye! Isekai Tensei
Scraped Title 9887/9900: Isekai Death Game ni Tensou sarete Tsurai
Scraped Title 9888/9900: xx demo Mahou Shoujo ni Naremasu ka?
Scraped Title 9889/9900: Hinomoto 3-shimai 

Scraped Title 10038/10100: Bradherley no Basha
Scraped Title 10039/10100: Kimi dake ni Ai wo.
Scraped Title 10040/10100: Samurai 7
Scraped Title 10041/10100: Sakura Yuki
Scraped Title 10042/10100: Try! Try! Try! Webcomics
Scraped Title 10043/10100: Hanayashiki no Juunin-tachi
Scraped Title 10044/10100: Eden
Scraped Title 10045/10100: Heroman
Scraped Title 10046/10100: Kansen Rettou
Scraped Title 10047/10100: Yume no Atosaki
Scraped Title 10048/10100: Agape
Scraped Title 10049/10100: Nigeru Otoko
Scraped Title 10050/10100: Kantan dakedo, Mendokusai
Scraped Title 10051/10100: Kanchigai no Atelier Meister: Eiyuu Party no Moto Zatsuyougakari ga, Jitsu wa Sentou Igai ga SSS Rank Datta to Iu Yoku Aru Hanashi
Scraped Title 10052/10100: A-Rank Boukensha no Slow Life
Scraped Title 10053/10100: Kuro Gal-san ga Kuru!
Scraped Title 10054/10100: Hangyaku no Soul Eater
Scraped Title 10055/10100: Classmate no Tanaka-san wa Sugoku Kowai
Scraped Title 10056/10100: Suterareta Tensei Kenja: Mamono no Mor

Loop 102
Scraped Title 10200/10300: Hime tte Yonde ne
Scraped Title 10201/10300: Specter
Scraped Title 10202/10300: Mangetsu Monogatari
Scraped Title 10203/10300: Aoi Haru
Scraped Title 10204/10300: Hana ni Nare
Scraped Title 10205/10300: Kiss kara Hajimeyou
Scraped Title 10206/10300: Love & Sex
Scraped Title 10207/10300: Junai Clover
Scraped Title 10208/10300: Mahou Tsukai Tai!
Scraped Title 10209/10300: Gokko
Scraped Title 10210/10300: Boku no Marie
Scraped Title 10211/10300: Tokyo Zombie
Scraped Title 10212/10300: Yokujou Kiss
Scraped Title 10213/10300: Jigoku no Komoriuta
Scraped Title 10214/10300: Shihonron
Scraped Title 10215/10300: Usotsuki Marriage
Scraped Title 10216/10300: Dokuro
Scraped Title 10217/10300: Zangekikan
Scraped Title 10218/10300: Ai ga Nakutemo Kutteyukemasu.
Scraped Title 10219/10300: Momoiro Punch
Scraped Title 10220/10300: Densen Complex
Scraped Title 10221/10300: Sweet Lovin' Baby
Scraped Title 10222/10300: 17-sai
Scraped Title 10223/10300: Match Ippon no Ha

Scraped Title 10364/10400: Anata no Mune ni Kaeru Hi made
Scraped Title 10365/10400: Hana no Miyako!
Scraped Title 10366/10400: Toscana no Yuuutsu
Scraped Title 10367/10400: Ra-i
Scraped Title 10368/10400: Koi ni Okite
Scraped Title 10369/10400: Pineapple Army
Scraped Title 10370/10400: Cinderella Boy
Scraped Title 10371/10400: Kore ga Watashi no Goshujinsama
Scraped Title 10372/10400: 23:00
Scraped Title 10373/10400: Happy★Lesson: Mama Sensei wa Saikou!
Scraped Title 10374/10400: Peridot
Scraped Title 10375/10400: Ai no Koe
Scraped Title 10376/10400: Shichinin no Nana
Scraped Title 10377/10400: Kimi no Tonari de Nemuritai
Scraped Title 10378/10400: Pretty in Blue
Scraped Title 10379/10400: Sister Red
Scraped Title 10380/10400: Choukyouai
Scraped Title 10381/10400: Desert Storm
Scraped Title 10382/10400: Doukyuusei H kara Hajimaru Koi
Scraped Title 10383/10400: Honey na Koto
Scraped Title 10384/10400: Perfect Partner
Scraped Title 10385/10400: Otome Cake
Scraped Title 10386/10400: Wedd

...

Scraped Title 11783/11800: Legend of Lemnear
Scraped Title 11784/11800: Benkyou Shinasai!
Scraped Title 11785/11800: Aishiatteru Futari
Scraped Title 11786/11800: Kyoko
Scraped Title 11787/11800: Eikou no Te
Scraped Title 11788/11800: Baby Pink KISS
Scraped Title 11789/11800: Virgin Wars
Scraped Title 11790/11800: Bokura no Fushidara
Scraped Title 11791/11800: Ayuta
Scraped Title 11792/11800: Comic Lily Plus
Scraped Title 11793/11800: I Am a Hero in Nagasaki
Scraped Title 11794/11800: Jinmen
Scraped Title 11795/11800: Dorei Yuugi
Scraped Title 11796/11800: Fuku Neko
Scraped Title 11797/11800: Tennen Kakutou Shoujo Chihiro-chan♥
Scraped Title 11798/11800: Aimane: Akuma na Kanojo wo Produce
Scraped Title 11799/11800: Tokyo Summer of the Dead
Loop 118
Scraped Title 11800/11900: Iczer Densetsu
Scraped Title 11801/11900: Hiki
Scraped Title 11802/11900: 0 kara Hajimeru Manga Kyoushitsu
Scraped Title 11803/11900: Junai Twitter
Scraped Title 11804/11900: Yagyuu Hijouken Samon
Scraped Title 118

Scraped Title 11944/12000: Cigarette Anthology
Scraped Title 11945/12000: Oingo to Boingo Kyoudai Daibouken
Scraped Title 11946/12000: Kono Shima ni wa Midara de Jaaku na Mono ga Sumu
Scraped Title 11947/12000: Kamen Boxer
Scraped Title 11948/12000: Tetsuo: The Bullet Man
Scraped Title 11949/12000: Comics Underground Japan
Scraped Title 11950/12000: Cross Days
Scraped Title 11951/12000: Underdog
Scraped Title 11952/12000: Romantic Egoist
Scraped Title 11953/12000: Sukeban Arashi
Scraped Title 11954/12000: Maji Bura!?
Scraped Title 11955/12000: Tonkaradani Monogatari
Scraped Title 11956/12000: Anorexia: Shikabane Hanako wa Kyoshokushou
Scraped Title 11957/12000: Aku no Joou no Kiseki
Scraped Title 11958/12000: Strike or Gutter
Scraped Title 11959/12000: Tokyo Shinobi Squad
Scraped Title 11960/12000: Sono Rettou Kishi, Level 999
Scraped Title 11961/12000: Tanaka no Atelier: Nenrei=Kanojo Inai Reki no Mahoutsukai
Scraped Title 11962/12000: Zombies
Scraped Title 11963/12000: Hana wa Nisemo

Scraped Title 12109/12200: Oni: 2889-nen no Hanran
Scraped Title 12110/12200: Full Drive
Scraped Title 12111/12200: Peach Milk Crown
Scraped Title 12112/12200: Kowloons' Ball Parade
Scraped Title 12113/12200: Kiss no Tochuu
Scraped Title 12114/12200: Mahou Onna ga Kuru
Scraped Title 12115/12200: Seikimatsu Occult Gakuin
Scraped Title 12116/12200: Aoi Tori no Shinwa: Blue Myth Overture
Scraped Title 12117/12200: Anamorphosis no Meijuu
Scraped Title 12118/12200: Itokoi Chidori
Scraped Title 12119/12200: Koisuru Edison
Scraped Title 12120/12200: Zettai Fukujuu Game
Scraped Title 12121/12200: Yugawara-kun wa Ooyamada Danshi Koukou de Moteru Houhou wo Kangaeteita ga
Scraped Title 12122/12200: Boku to Roboko
Scraped Title 12123/12200: Fushigi no Kuni no Miyuki-chan
Scraped Title 12124/12200: Burn-Up: Excess & W
Scraped Title 12125/12200: Yuutai Nova
Scraped Title 12126/12200: MÄR Omega
Scraped Title 12127/12200: Aibu de Jirashite
Scraped Title 12128/12200: Waza no Tabibito
Scraped Title 1212

Scraped Title 12271/12300: Bozebeats
Scraped Title 12272/12300: Houkago Bitchcraft
Scraped Title 12273/12300: Shuumatsu no Harem: Britannia Lumiére
Scraped Title 12274/12300: Kawari Nadeshiko: 3 Shimai no Himitsu
Scraped Title 12275/12300: Mainichi Shine Shine Itte Kuru Gimai ga, Ore ga Neteiru Suki ni Saiminjutsu de Horesaseyou toshite Kuru n desu kedo......!
Scraped Title 12276/12300: Uwa-Koi
Scraped Title 12277/12300: Takayukashiki Shoujo
Scraped Title 12278/12300: Lolicon Saga
Scraped Title 12279/12300: Cambrian
Scraped Title 12280/12300: Oshiri Sawaritagaru Hito Nan nano
Scraped Title 12281/12300: Cutie Honey Tai Devilman Lady
Scraped Title 12282/12300: Haru Hayate
Scraped Title 12283/12300: Elf Yome to Hajimeru Isekai Ryoushu Seikatsu
Scraped Title 12284/12300: Bakumatsu Roman Ibun
Scraped Title 12285/12300: Lady Justice
Scraped Title 12286/12300: Samurai 8: Hachimaru Den
Scraped Title 12287/12300: ⅒ no Hanayome
Scraped Title 12288/12300: Mad Summer School
Scraped Title 12289/123

Scraped Title 12441/12500: Noel Evolution
Scraped Title 12442/12500: PPOI!
Scraped Title 12443/12500: Toe Shoes
Scraped Title 12444/12500: Tokumu Sentai Shinesman
Scraped Title 12445/12500: Kikaider 02
Scraped Title 12446/12500: Himesama Ninpouchou: Tenka☆Musou
Scraped Title 12447/12500: Go Go Heaven!!
Scraped Title 12448/12500: Asaki Yumemishi: Genji Monogatari
Scraped Title 12449/12500: Love Song
Scraped Title 12450/12500: Datte Onnanoko no yoo
Scraped Title 12451/12500: Wild Com.
Scraped Title 12452/12500: Top wo Nerae 2!
Scraped Title 12453/12500: Gal Samurai
Scraped Title 12454/12500: Vaizard
Scraped Title 12455/12500: Gunparade March
Scraped Title 12456/12500: Chishiryou Dolis
Scraped Title 12457/12500: Mystical Prince Yoshida-kun!
Scraped Title 12458/12500: Kagerou-Nostalgia
Scraped Title 12459/12500: My Home Mirano
Scraped Title 12460/12500: Chou Kidou Bouhatsu Shuukyuu Yarou: Libero no Takeda
Scraped Title 12461/12500: Macross Dynamite: 7 Mylene Beat
Scraped Title 12462/12500:

### Webscrap User Review For Each Series

In [3]:
# Read manga list from file
manga_list = pd.read_csv("../data/manga_titles.csv")
manga_list.drop(columns=['Unnamed: 0'], inplace=True)

In [4]:
start = 1000
end = 1500
#end = manga_list.shape[0]
no_of_pages = 20
for index in range(start, end, 1):
    title = manga_list.loc[index, 'manga_title']
    base_url = manga_list.loc[index, 'manga_link']
    
    # Initialize
    reviews = {'title': list(), 'member':list(), 'score':list(), 'status':list(), 'vol_read':list(), 'chpt_read':list(), 'member_link':list()}
    
    # Monitor for code 404
    code_404 = False
    
    for page in range(no_of_pages):
        # Set origin url
        url = base_url + '/stats?show=' + str(page*75)

        # Set initial status
        status = 0

        while status != 200 and not code_404:
            try:
                res = requests.get(url)
                status = res.status_code
                if status == 200:
                    scrape_user_stats_from_one_manga(res, reviews, title)
                    print(f"Scraped Users' Score from {title}: Page {page}")
                elif status == 404:
                    code_404 = True
                    print(f"Code {status} occurred at index {page}")
                    break
                else:
                    print(f"Code {status} occurred at index {page}")

                time.sleep(random.random() * 5)
            
            except Exception as e:
                print(e)

    # Change dict to DataFrame
    reviews_pd = pd.DataFrame.from_dict(reviews)
    
    # Save DataFrame to CSV
    reviews_pd.to_csv(f"../data/users_scores/{index}.csv")
    reviews_pd
    
    # Wait for 30 seconds
    time.sleep(30)

Scraped Users' Score from JoJo no Kimyou na Bouken: Crazy Diamond no Akuryou-teki Shitsuren: Page 0
Scraped Users' Score from JoJo no Kimyou na Bouken: Crazy Diamond no Akuryou-teki Shitsuren: Page 1
Scraped Users' Score from JoJo no Kimyou na Bouken: Crazy Diamond no Akuryou-teki Shitsuren: Page 2
Scraped Users' Score from JoJo no Kimyou na Bouken: Crazy Diamond no Akuryou-teki Shitsuren: Page 3
Scraped Users' Score from JoJo no Kimyou na Bouken: Crazy Diamond no Akuryou-teki Shitsuren: Page 4
Scraped Users' Score from JoJo no Kimyou na Bouken: Crazy Diamond no Akuryou-teki Shitsuren: Page 5
Scraped Users' Score from JoJo no Kimyou na Bouken: Crazy Diamond no Akuryou-teki Shitsuren: Page 6
Scraped Users' Score from JoJo no Kimyou na Bouken: Crazy Diamond no Akuryou-teki Shitsuren: Page 7
Scraped Users' Score from JoJo no Kimyou na Bouken: Crazy Diamond no Akuryou-teki Shitsuren: Page 8
Scraped Users' Score from JoJo no Kimyou na Bouken: Crazy Diamond no Akuryou-teki Shitsuren: Page 9


Scraped Users' Score from Fate/Zero: Page 0
Scraped Users' Score from Fate/Zero: Page 1
Scraped Users' Score from Fate/Zero: Page 2
Scraped Users' Score from Fate/Zero: Page 3
Scraped Users' Score from Fate/Zero: Page 4
Scraped Users' Score from Fate/Zero: Page 5
Scraped Users' Score from Fate/Zero: Page 6
Scraped Users' Score from Fate/Zero: Page 7
Scraped Users' Score from Fate/Zero: Page 8
Scraped Users' Score from Fate/Zero: Page 9
Scraped Users' Score from Fate/Zero: Page 10
Scraped Users' Score from Fate/Zero: Page 11
Scraped Users' Score from Fate/Zero: Page 12
Scraped Users' Score from Fate/Zero: Page 13
Scraped Users' Score from Fate/Zero: Page 14
Scraped Users' Score from Fate/Zero: Page 15
Scraped Users' Score from Fate/Zero: Page 16
Scraped Users' Score from Fate/Zero: Page 17
Scraped Users' Score from Fate/Zero: Page 18
Scraped Users' Score from Fate/Zero: Page 19
Scraped Users' Score from Gokusen Kanketsu-hen: Page 0
Scraped Users' Score from Gokusen Kanketsu-hen: Page 1


Scraped Users' Score from Motto Ganbare! Nakamura-kun!!: Page 13
Scraped Users' Score from Motto Ganbare! Nakamura-kun!!: Page 14
Scraped Users' Score from Motto Ganbare! Nakamura-kun!!: Page 15
Scraped Users' Score from Motto Ganbare! Nakamura-kun!!: Page 16
Scraped Users' Score from Motto Ganbare! Nakamura-kun!!: Page 17
Scraped Users' Score from Motto Ganbare! Nakamura-kun!!: Page 18
Scraped Users' Score from Motto Ganbare! Nakamura-kun!!: Page 19
Scraped Users' Score from Sensei wa Koi wo Oshierarenai: Page 0
Scraped Users' Score from Sensei wa Koi wo Oshierarenai: Page 1
Scraped Users' Score from Sensei wa Koi wo Oshierarenai: Page 2
Scraped Users' Score from Sensei wa Koi wo Oshierarenai: Page 3
Scraped Users' Score from Sensei wa Koi wo Oshierarenai: Page 4
Scraped Users' Score from Sensei wa Koi wo Oshierarenai: Page 5
Scraped Users' Score from Sensei wa Koi wo Oshierarenai: Page 6
Scraped Users' Score from Sensei wa Koi wo Oshierarenai: Page 7
Scraped Users' Score from Sensei 

Scraped Users' Score from Urusei Yatsura: Page 4
Scraped Users' Score from Urusei Yatsura: Page 5
Scraped Users' Score from Urusei Yatsura: Page 6
Scraped Users' Score from Urusei Yatsura: Page 7
Scraped Users' Score from Urusei Yatsura: Page 8
Scraped Users' Score from Urusei Yatsura: Page 9
Scraped Users' Score from Urusei Yatsura: Page 10
Scraped Users' Score from Urusei Yatsura: Page 11
Scraped Users' Score from Urusei Yatsura: Page 12
Scraped Users' Score from Urusei Yatsura: Page 13
Scraped Users' Score from Urusei Yatsura: Page 14
Scraped Users' Score from Urusei Yatsura: Page 15
Scraped Users' Score from Urusei Yatsura: Page 16
Scraped Users' Score from Urusei Yatsura: Page 17
Scraped Users' Score from Urusei Yatsura: Page 18
Scraped Users' Score from Urusei Yatsura: Page 19
Scraped Users' Score from Tajuu Jinkaku Tantei Psycho: Page 0
Scraped Users' Score from Tajuu Jinkaku Tantei Psycho: Page 1
Scraped Users' Score from Tajuu Jinkaku Tantei Psycho: Page 2
Scraped Users' Score

Scraped Users' Score from Suzumiya Haruhi-chan no Yuuutsu: Page 15
Scraped Users' Score from Suzumiya Haruhi-chan no Yuuutsu: Page 16
Scraped Users' Score from Suzumiya Haruhi-chan no Yuuutsu: Page 17
Scraped Users' Score from Suzumiya Haruhi-chan no Yuuutsu: Page 18
Scraped Users' Score from Suzumiya Haruhi-chan no Yuuutsu: Page 19
Scraped Users' Score from Flat: Page 0
Scraped Users' Score from Flat: Page 1
Scraped Users' Score from Flat: Page 2
Scraped Users' Score from Flat: Page 3
Scraped Users' Score from Flat: Page 4
Scraped Users' Score from Flat: Page 5
Scraped Users' Score from Flat: Page 6
Scraped Users' Score from Flat: Page 7
Scraped Users' Score from Flat: Page 8
Scraped Users' Score from Flat: Page 9
Scraped Users' Score from Flat: Page 10
Scraped Users' Score from Flat: Page 11
Scraped Users' Score from Flat: Page 12
Scraped Users' Score from Flat: Page 13
Scraped Users' Score from Flat: Page 14
Scraped Users' Score from Flat: Page 15
Scraped Users' Score from Flat: Pag

...

Scraped Users' Score from Youkai Apartment no Yuuga na Nichijou: Page 16
Scraped Users' Score from Youkai Apartment no Yuuga na Nichijou: Page 17
Scraped Users' Score from Youkai Apartment no Yuuga na Nichijou: Page 18
Scraped Users' Score from Youkai Apartment no Yuuga na Nichijou: Page 19
Scraped Users' Score from Hunter x Hunter: Kurapika Tsuioku-hen: Page 0
Scraped Users' Score from Hunter x Hunter: Kurapika Tsuioku-hen: Page 1
Scraped Users' Score from Hunter x Hunter: Kurapika Tsuioku-hen: Page 2
Scraped Users' Score from Hunter x Hunter: Kurapika Tsuioku-hen: Page 3
Scraped Users' Score from Hunter x Hunter: Kurapika Tsuioku-hen: Page 4
Scraped Users' Score from Hunter x Hunter: Kurapika Tsuioku-hen: Page 5
Scraped Users' Score from Hunter x Hunter: Kurapika Tsuioku-hen: Page 6
Scraped Users' Score from Hunter x Hunter: Kurapika Tsuioku-hen: Page 7
Scraped Users' Score from Hunter x Hunter: Kurapika Tsuioku-hen: Page 8
Scraped Users' Score from Hunter x Hunter: Kurapika Tsuioku-

Scraped Users' Score from Sannensa: Page 18
Scraped Users' Score from Sannensa: Page 19
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 0
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 1
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 2
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 3
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 4
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 5
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 6
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 7
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 8
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 9
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 10
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 11
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 12
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 13
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 14
Scraped Users' Score from Sen wa, Boku wo Egaku: Page 15
Scraped Us

Scraped Users' Score from Tenkaichi: Nihon Saikyou Bugeisha Ketteisen: Page 14
Scraped Users' Score from Tenkaichi: Nihon Saikyou Bugeisha Ketteisen: Page 15
Scraped Users' Score from Tenkaichi: Nihon Saikyou Bugeisha Ketteisen: Page 16
Scraped Users' Score from Tenkaichi: Nihon Saikyou Bugeisha Ketteisen: Page 17
Scraped Users' Score from Tenkaichi: Nihon Saikyou Bugeisha Ketteisen: Page 18
Scraped Users' Score from Tenkaichi: Nihon Saikyou Bugeisha Ketteisen: Page 19
Scraped Users' Score from Kimi no Knife: Page 0
Scraped Users' Score from Kimi no Knife: Page 1
Scraped Users' Score from Kimi no Knife: Page 2
Scraped Users' Score from Kimi no Knife: Page 3
Scraped Users' Score from Kimi no Knife: Page 4
Scraped Users' Score from Kimi no Knife: Page 5
Scraped Users' Score from Kimi no Knife: Page 6
Scraped Users' Score from Kimi no Knife: Page 7
Scraped Users' Score from Kimi no Knife: Page 8
Scraped Users' Score from Kimi no Knife: Page 9
Scraped Users' Score from Kimi no Knife: Page 

Scraped Users' Score from K-On! College: Page 11
Scraped Users' Score from K-On! College: Page 12
Scraped Users' Score from K-On! College: Page 13
Scraped Users' Score from K-On! College: Page 14
Scraped Users' Score from K-On! College: Page 15
Scraped Users' Score from K-On! College: Page 16
Scraped Users' Score from K-On! College: Page 17
Scraped Users' Score from K-On! College: Page 18
Scraped Users' Score from K-On! College: Page 19
Scraped Users' Score from Pretty Face: Page 0
Scraped Users' Score from Pretty Face: Page 1
Scraped Users' Score from Pretty Face: Page 2
Scraped Users' Score from Pretty Face: Page 3
Scraped Users' Score from Pretty Face: Page 4
Scraped Users' Score from Pretty Face: Page 5
Scraped Users' Score from Pretty Face: Page 6
Scraped Users' Score from Pretty Face: Page 7
Scraped Users' Score from Pretty Face: Page 8
Scraped Users' Score from Pretty Face: Page 9
Scraped Users' Score from Pretty Face: Page 10
Scraped Users' Score from Pretty Face: Page 11
Scrap

Scraped Users' Score from Minami-ke: Page 1
Scraped Users' Score from Minami-ke: Page 2
Scraped Users' Score from Minami-ke: Page 3
Scraped Users' Score from Minami-ke: Page 4
Scraped Users' Score from Minami-ke: Page 5
Scraped Users' Score from Minami-ke: Page 6
Scraped Users' Score from Minami-ke: Page 7
Scraped Users' Score from Minami-ke: Page 8
Scraped Users' Score from Minami-ke: Page 9
Scraped Users' Score from Minami-ke: Page 10
Scraped Users' Score from Minami-ke: Page 11
Scraped Users' Score from Minami-ke: Page 12
Scraped Users' Score from Minami-ke: Page 13
Scraped Users' Score from Minami-ke: Page 14
Scraped Users' Score from Minami-ke: Page 15
Scraped Users' Score from Minami-ke: Page 16
Scraped Users' Score from Minami-ke: Page 17
Scraped Users' Score from Minami-ke: Page 18
Scraped Users' Score from Minami-ke: Page 19
Scraped Users' Score from LifE: Page 0
Scraped Users' Score from LifE: Page 1
Scraped Users' Score from LifE: Page 2
Scraped Users' Score from LifE: Page 